In [ ]:
import itertools
import os

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np
import pandas as pd
from scipy import spatial

from utils import estimate_normals, normals_to_rgb, sph2cart
from plotting import set_axes_equal, set_defense_context, draw_unit_cube

In [ ]:
%config InlineBackend.figure_format = 'retina'

# Planar model

Figure 2. Skin models for dosimetry analysis: (a) skin models with  different tissue compositions; (b) exposure condition.

K. Li et al., "Intercomparison of Calculated Incident Power Density and Temperature Rise for Exposure From Different Antennas at 10–90 GHz," in IEEE Access, vol. 9, pp. 151654-151666, 2021, doi: 10.1109/ACCESS.2021.3126738.

# Spherical model

In [ ]:
# generate linearly spaced point cloud in spherical coordinates

r = 1
theta = np.linspace(0, 2 * np.pi, 31)
phi = np.linspace(0, np.pi, 13)
Theta, Phi = np.meshgrid(theta, phi)
x, y, z = sph2cart(r, Theta.ravel(), Phi.ravel())
xyz = np.c_[x, y, z]

In [ ]:
# create convex hull and extract the surface area

hull = spatial.ConvexHull(xyz)
A_hull = hull.area
print(f'A = {A_hull:.6f}')

In [ ]:
# estimate normals

n = estimate_normals(xyz, orient=True)

In [ ]:
# visualize normals as arrows

with set_defense_context():
    fig = plt.figure(figsize=(3, 3), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    ax.scatter(*xyz.T, c='k', s=0.5)
    ax.quiver(*xyz.T, *n.T, color='k', length=0.25, lw=0.5)
    hull_triangle_coords = hull.points[hull.simplices]
    triangles = Poly3DCollection(hull_triangle_coords,
                                 ec='k', fc='gray', lw=0.25, alpha=0.25)
    ax.add_collection3d(triangles)
    ax.view_init(30, 60)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = '00-spherical-tissue-model-quiver.png'
    # fig.savefig(os.path.join('figures', fname), dpi=500, bbox_inches='tight')

In [ ]:
# convert arrows to rgb cube

c = normals_to_rgb(n)

with set_defense_context():
    fig = plt.figure(figsize=(3, 3), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz.T, c=c, s=10)
    ax.view_init(30, 60)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = '00-spherical-tissue-model-rgb.png'
    # fig.savefig(os.path.join('figures', fname), dpi=500, bbox_inches='tight')

# Cylindrical model

In [ ]:
# generate linearly spaced point cloud in polar coordinates
n = 24
assert n % 3 == 0, 'n must be divisible by 3'
t = np.linspace(0, 2 * np.pi, n, endpoint=False)
x = r * np.cos(t)
y = r * np.sin(t)
xy = np.c_[x, y]
xyz = np.c_[np.repeat(xy, int(n/3), axis=0),
      np.tile(np.linspace(-1, 1, int(n/3)), n)]

In [ ]:
# create smooth spherical surface

hull = spatial.ConvexHull(xyz)
A_hull = hull.area
print(f'A = {A_hull:.6f}')

In [ ]:
# estimate normals

n = estimate_normals(xyz, orient=True)

In [ ]:
# visualize normals as arrows

with set_defense_context():
    fig = plt.figure(figsize=(3, 3), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    ax.scatter(*xyz.T, c='k', s=0.5)
    ax.quiver(*xyz.T, *n.T, color='k', length=0.25, lw=0.5)
    hull_triangle_coords = hull.points[hull.simplices]
    triangles = Poly3DCollection(hull_triangle_coords,
                                 ec='k', fc='gray', lw=0.25, alpha=0.25)
    ax.add_collection3d(triangles)
    ax.view_init(30, 60)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    fig.tight_layout()
    plt.show()
    fname = '00-cylindrical-tissue-model-quiver.png'
    # fig.savefig(os.path.join('figures', fname), dpi=500, bbox_inches='tight')

In [ ]:
# convert arrows to rgb cube

c = normals_to_rgb(n)

with set_defense_context():
    fig = plt.figure(figsize=(3, 3), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz.T, c=c, s=10)
    ax.view_init(30, 60)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = '00-cylindrical-tissue-model-rgb.png'
    # fig.savefig(os.path.join('figures', fname), dpi=500, bbox_inches='tight')

# RGB cube encoding normals

In [ ]:
# set up coloring

pts = np.array(list(itertools.product([0, 1], repeat=3)))
cs = ['black', 'blue', 'green', 'cyan', 'red', 'magenta', 'yellow', 'white']
pairs = pd.DataFrame(data=pts, columns=['x', 'y', 'z'])
pairs['cs'] = cs

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(2, 2), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    ax = draw_unit_cube(ax)
    ax.scatter(*pts.T, c=cs, edgecolor='k', depthshade=False, s=400)
    ax.view_init(30, 60)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    fig.tight_layout()
    plt.show()
    fname = '00-rgb-cube.png'
    # fig.savefig(os.path.join('figures', fname), dpi=500, bbox_inches='tight')